# Beautiful Soup Scripts - GET Request

*To Run - Open file in Jupyter Notebook - Kernel -> Restar and Run All*

**This book contains a simple example of a Beautiful Soup GET Request** 


1. A simple bot to collect Trader Joes location. I used this data to perform a simple visual hypothesis - **Trader Joe's is available only in locations which cater to high-income earners**. I did this analysis as a side project and just for the state of Massachusetts.

2. I used these locations to reverse geocode and find the latitude and longitudes to access all the locations across Massachusetts. 

3. Open the `Sample_Application.html` in the folder to see how I used this Data

*It is important to note that before any Web Scraping task it is required to check the website's Robot.txt*

In [1]:
'''

Importing Required Packages

'''
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time
import os
wd = os.getcwd()

In [2]:
#Specifying the URL
url = 'https://locations.traderjoes.com'
r = requests.get(url, verify=False)

#Getting all links 
soup = BeautifulSoup(r.text, 'html.parser')
state_link = soup.findAll('a',{'class':'ga_w2gi_lp listitem'})

#Initialising an empty Dataframe 
data = pd.DataFrame()

#Minimising for just the state of Massachusetts
state_links = soup.findAll('a',{'class':'ga_w2gi_lp listitem'})

for sl in state_links:
    s_link = url+sl.get("href")
    s_name = sl.get('data-galoc')
    
    #Condition to Capture the required State
    if "Massachusetts" in s_name:
        #Adding a sleep of 2 seconds
        time.sleep(2)

        r = requests.get(s_link, verify = False)

        soup = BeautifulSoup(r.text, 'html.parser')

        c_link = soup.findAll('a',{'class':'ga_w2gi_lp listitem'})

        c_link = [url+x.get("href") for x in c_link]
        for cl in c_link:
            r = requests.get(cl, verify=False,timeout=(3.05, 27))
            soup = BeautifulSoup(r.text, 'html.parser')
            time.sleep(2)

            #Extracting data from the Json embeddeed in soup
            json_content = json.loads(soup.findAll('script',{"type":'application/ld+json'})[1].contents[0])

            #Directly storing as a Pandas DataFrame
            df = pd.DataFrame(json_content['address'],index=[0])
            df['URL'] = cl
            df['Name'] = json_content['name']
            df['State'] = s_name
            data = pd.concat([data,df])
            #print(s_name,json_content['name'])

print(data.head())            
#data.to_csv(wd+"\\TJ_Location.csv",index=False)

C:\Users\abhis\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'locations.traderjoes.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\abhis\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'locations.traderjoes.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\abhis\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'locations.traderjoes.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/e

           @type             streetAddress addressLocality addressRegion  \
0  PostalAddress              145 Great Rd           Acton            MA   
0  PostalAddress  199 North Harvard Street         Allston            MA   
0  PostalAddress    1427 Massachusetts Ave       Arlington            MA   
0  PostalAddress           500 Boylston St          Boston            MA   
0  PostalAddress            1317 Beacon St       Brookline            MA   

  postalCode addressCountry                                             URL  \
0      01720             US      https://locations.traderjoes.com/ma/acton/   
0      02134             US    https://locations.traderjoes.com/ma/allston/   
0      02476             US  https://locations.traderjoes.com/ma/arlington/   
0      02116             US     https://locations.traderjoes.com/ma/boston/   
0      02446             US  https://locations.traderjoes.com/ma/brookline/   

                                           Name               State 